In [3]:
import os
from typing import Annotated, TypedDict, List, Literal

from langchain_core.tools import tool
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_agent

from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

In [4]:
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
load_dotenv()
llm = init_chat_model("claude-sonnet-4-5-20250929")

In [ ]:
class AgentState(MessagesState):
    next_agent: str # which agent to call next

In [9]:
# create simple tools
@tool
def search_web(query: str) -> str:
    """Search the web using Tavily Search."""
    search = TavilySearchResults(max_results=3)
    results = search.invoke(query)
    return str(results)

@tool
def write_summary(content: str) -> str:
    """Write a brief summary of the given content."""
    summary = f"This is a brief summary of the content: {content[:75]}..."
    return summary

In [10]:
# define agent functions (simple approach)
def researcher_agent(state: AgentState):
    """Researcher agent that searches the web for information."""

    messages = state['messages']

    # add system message for context
    system_msg = SystemMessage(content="You are a researcher agent. Your task is to use search_web tool to gather information on a given topic.")

    # call llm with tools
    researcher_llm = llm.bind_tools([search_web])
    response = researcher_llm.invoke([system_msg] + messages)

    # return the response and route to writer agent
    return {
        'messages': [response],
        'next_agent': 'writer'
    }

In [11]:
def writer_agent(state: AgentState):
    """Writer agent that summarizes the researched information."""

    messages = state['messages']

    # add system message for context
    system_msg = SystemMessage(content="You are a writer agent. Your task is to use write_summary tool to summarize the researched information.")

    # call llm with tools
    writer_llm = llm.bind_tools([write_summary])
    response = writer_llm.invoke([system_msg] + messages)

    # return the response and end the process
    return {
        'messages': [response],
        'next_agent': 'end'
    }

In [12]:
# tool executor node
def execute_tools(state: AgentState):
    """Execute any pending tool calls"""
    messages = state['messages']
    last_message = messages[-1]

    # check if there are tool calls to execute
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        # create tool node and execute tools
        tool_node = ToolNode(tools=[search_web, write_summary])
        response = tool_node.invoke(state)
        return response

    # no tool calls, return state as is
    return state